In [9]:
spark.sparkContext.stop

In [10]:
import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark._
import org.apache.spark.rdd.RDD
import spark.implicits._


val SPARK_NAMESPACE="default"
val SA="spark-driver"
val K8S_CACERT="/var/run/secrets/kubernetes.io/serviceaccount/ca.crt"
val K8S_TOKEN="/var/run/secrets/kubernetes.io/serviceaccount/token"
val DOCKER_IMAGE="hishailesh77/spark_3_0_1:latest"
val SPARK_DRIVER_HOST= "jupyter-lab-host.jupyter-headless.default.svc.cluster.local"
val SPARK_DRIVER_PORT="20020"

import org.apache.spark.sql.{DataFrame, SparkSession}
import org.apache.spark._
import org.apache.spark.rdd.RDD
import spark.implicits._
SPARK_NAMESPACE: String = default
SA: String = spark-driver
K8S_CACERT: String = /var/run/secrets/kubernetes.io/serviceaccount/ca.crt
K8S_TOKEN: String = /var/run/secrets/kubernetes.io/serviceaccount/token
DOCKER_IMAGE: String = hishailesh77/spark2.4.6-deb:latest
SPARK_DRIVER_HOST: String = jupyter-lab-host.jupyter-headless.default.svc.cluster.local
SPARK_DRIVER_PORT: String = 20020


In [11]:
val spark = SparkSession
.builder()
.appName("Spark-Spylon")
.master("k8s://https://kubernetes.default:443")
.config("spark.kubernetes.authenticate.driver.serviceAccountName",SA)
.config("spark.kubernetes.namespace",SPARK_NAMESPACE)
.config("spark.kubernetes.authenticate.subdmission.caCertFile",K8S_CACERT)
.config("spark.kubernetes.authenticate.submission.oauthTokenFile",K8S_TOKEN)
.config("spark.kubernetes.container.image", DOCKER_IMAGE)
.config("spark.kubernetes.container.image.pullPolicy","Always")
.config("spark.driver.port",SPARK_DRIVER_PORT)
.config("spark.driver.host",SPARK_DRIVER_HOST)
.config("spark.executor.instances", "12")
.config("spark.driver.memory","16g")
.config("spark.executor.memory","16g")
.config("spark.driver.cores","8")
.config("spark.executor.cores","4")
.config("spark.memory.offHeap.enabled","true")
.config("spark.memory.offHeap.size","8g")
.config("spark.hadoop.fs.s3a.access.key", "")
.config("spark.hadoop.fs.s3a.secret.key", "")
.config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
.config("spark.hadoop.fs.s3a.multiobjectdelete.enable","false")
.config("spark.hadoop.fs.s3a.fast.upload","true")
.getOrCreate()

spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@511947b2


In [12]:
import spark.implicits._
import spark.sql

val df_1 = spark.read.format("parquet").option("inferSchema", "true").
    option("header", "false").
    load("s3a://ml-workflow-data/security/Malware_Dataset/Output/malware_tmp_parquet_1")

import spark.implicits._
import spark.sql
df_1: org.apache.spark.sql.DataFrame = [file_name: string, File: string ... 2 more fields]


In [18]:
df_1.rdd.getNumPartitions

res5: Int = 51


In [19]:
val df_2 = df_1.repartition(5000)

df_2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [file_name: string, File: string ... 2 more fields]


In [20]:
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{CountVectorizer, RegexTokenizer, StopWordsRemover, VectorAssembler}
import org.apache.spark.sql.{DataFrame, SparkSession}
def featureEngineeringCountV(dataMatrix: DataFrame): DataFrame ={

      val tokenizer = new RegexTokenizer().setInputCol("Text").setOutputCol("wordsArray")
      val remover = new StopWordsRemover().setInputCol("wordsArray").setOutputCol("filteredWords")
      val cnt = new  CountVectorizer().setInputCol("filteredWords").setOutputCol("countv").setVocabSize(20)
      val assembler = new VectorAssembler().setInputCols(Array("countv")).setOutputCol("features")
      val pipeline = new Pipeline().setStages(Array(tokenizer, remover, cnt, assembler))
      val model= pipeline.fit(dataMatrix)
      model.transform(dataMatrix).drop("Text","wordsArray","filteredWords")

}

import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.feature.{CountVectorizer, RegexTokenizer, StopWordsRemover, VectorAssembler}
import org.apache.spark.sql.{DataFrame, SparkSession}
featureEngineeringCountV: (dataMatrix: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame


In [21]:
val final_df = featureEngineeringCountV(df_2)

final_df: org.apache.spark.sql.DataFrame = [file_name: string, File: string ... 3 more fields]


In [22]:
final_df.coalesce(100).write.mode("overwrite").format("parquet").option("compression", "snappy").mode("overwrite").save("s3a://ml-workflow-data/security/Malware_Dataset/Output/malware_tmp_parquet_2")

In [24]:
val test_df=spark.read.format("parquet").option("inferSchema", "true").
    option("header", "false").
    load("s3a://ml-workflow-data/security/Malware_Dataset/Output/malware_tmp_parquet_2")

test_df: org.apache.spark.sql.DataFrame = [file_name: string, File: string ... 3 more fields]


In [25]:
test_df.show()

+--------------------+--------------------+-----+--------------------+--------------------+
|           file_name|                File|label|              countv|            features|
+--------------------+--------------------+-----+--------------------+--------------------+
|13YpdP5vTLOazSQFRgJn|s3a://ml-workflow...|    3|(20,[0,1,3,4,5,6,...|[1094.0,564.0,0.0...|
|12tjh4qCkcHpObVBEeMr|s3a://ml-workflow...|    3|(20,[0,1,2,3,4,5,...|[2843.0,2300.0,86...|
|15loeAHtkJa8BuFi6Zry|s3a://ml-workflow...|    1|(20,[0,1,2,3,4,5,...|[126025.0,107847....|
|4JGbOVQnEt3ZP5acW7Yz|s3a://ml-workflow...|    6|(20,[0,1,2,3,4,5,...|[146066.0,130765....|
|5YMeDkHjclrCPd8OuymR|s3a://ml-workflow...|    4|(20,[0,1,2,3,4,5,...|[76734.0,68712.0,...|
|12Jd4qpOzTtQC3E6PXDb|s3a://ml-workflow...|    3|(20,[0,1,3,4,5,6,...|[1095.0,561.0,0.0...|
|7KqfVlEBOmr6tTQewNG8|s3a://ml-workflow...|    7|(20,[0,1,2,3,4,5,...|[2387.0,1167.0,48...|
|5PBvwNE8sCzm1bjUlf6A|s3a://ml-workflow...|    7|(20,[0,1,2,3,4,5,...|[2384.0,11

In [ ]:
test_df.